<center><h1>Time Series Part I</h1></center>
<br>
<br>

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

**References**:

[Iterating through directories with Python](https://stackoverflow.com/questions/19587118/iterating-through-directories-with-python)

[Read multiple csv files from multiple folders in Python](https://stackoverflow.com/questions/72663553/read-multiple-csv-files-from-multiple-folders-in-python)

[Pandas Doc for read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)

Package imports

In [1]:
# import python libraries
import pandas as pd
import statistics
import os
import numpy as np
from scipy import stats

Get the AReM Data Set

In [2]:
# try and play with it
df_b1_1 = pd.read_csv("../../data/AReM/bending1/dataset1.csv", header = 5, names = ['time','avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23'])
df_b1_1
# print(df_b1_1.avg_rss12)
mean1 = statistics.mean(df_b1_1.avg_rss12)
print("mean1 = ", mean1)

fstQ = statistics.quantiles(df_b1_1.avg_rss12, n=4)
print(fstQ[0])

mean1 =  40.6276617954071
39.25


In [3]:
df_b2_4 = pd.read_csv("../../data/AReM/bending2/dataset4.csv", header = 5, names = ['time','avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23'])
df_b2_4.head()  # Here I spotted a major error.

,time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,250 32.50 0.50 0.00 0.00 18.50 0.50,NaN,NaN,NaN,NaN,NaN,NaN
1,500 32.75 0.43 1.00 0.00 18.00 0.00,NaN,NaN,NaN,NaN,NaN,NaN
2,750 32.50 0.50 0.00 0.00 17.50 0.50,NaN,NaN,NaN,NaN,NaN,NaN
3,1000 32.50 0.50 7.50 0.50 17.50 0.87,NaN,NaN,NaN,NaN,NaN,NaN
4,1250 32.67 0.47 11.00 1.00 16.75 0.83,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# After examine the data, I found it's the problem of the separator being tampered with. It should be ',' but now it's a space
# Therefore, handle using the following method:
df_b2_4 = pd.read_csv("../../data/AReM/bending2/dataset4.csv", header = 5, names = ['time','avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23'], sep = ' ', index_col = False)
df_b2_4.head()  # Now it'a fine

# reference:
# https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

,time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,250,32.50,0.50,0.0,0.0,18.50,0.50
1,500,32.75,0.43,1.0,0.0,18.00,0.00
2,750,32.50,0.50,0.0,0.0,17.50,0.50
3,1000,32.50,0.50,7.5,0.5,17.50,0.87
4,1250,32.67,0.47,11.0,1.0,16.75,0.83


In [5]:
# deprecated
# try read files in one single directory
path = '../../data/AReM/bending1/'
df_list = []

for file in os.listdir(path):
    # Check if the file is a CSV (because there are pdf files exist)
    if file.endswith('.csv'):
        try:
            df = pd.read_csv(f'{path}{file}', 
                             header = 5, 
                             names = ['time','avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23'])
            df_list.append(df)
        except Exception as e:
            print(f"Error reading file {file}: {str(e)}")   # fail (qaq)

print(df_list[0])

       time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  var_rss23
0       250      42.00       0.00      18.00       0.00      11.33       0.94
1       500      42.75       0.43      16.75       1.79      18.25       0.43
2       750      42.50       0.50      16.75       0.83      19.00       1.22
3      1000      43.00       0.82      16.25       0.83      18.00       0.00
4      1250      43.67       0.47      12.75       0.83      19.00       0.82
..      ...        ...        ...        ...        ...        ...        ...
474  118750      44.25       0.83      16.25       1.30      24.00       0.00
475  119000      44.00       0.71      11.00       1.00      24.00       0.00
476  119250      44.25       0.43      10.25       1.30      22.75       0.83
477  119500      43.75       0.43       7.33       2.87      21.50       0.50
478  119750      44.50       0.50       3.00       1.22      21.50       0.50

[479 rows x 7 columns]


In [6]:
# reading files in several subdirectories
# get all file names
rootdir = '../../data/AReM/'
all_files = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        # Check if the file is a CSV (because there are pdf files exist)
        if file.endswith('.csv'):
            all_files.append(os.path.join(subdir, file))

print(len(all_files))

# reference: https://stackoverflow.com/questions/19587118/iterating-through-directories-with-python

# read all 88 data sets
df_list = {}    # changed to dictionary for train and test split

for file in all_files:
    try:
        if (file == "../../data/AReM/bending2/dataset4.csv"):
            df_temp = pd.read_csv(file, 
                                header = 5, 
                                names = ['time','avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23'], 
                                sep = ' ', 
                                index_col = False)
        else:
            df_temp = pd.read_csv(file, 
                                header = 5, 
                                names = ['time','avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23'])
        df_list[file] = df_temp
    except Exception as e:
        print(f"Error reading file {file}: {str(e)}")   # fail (qaq)

print(len(df_list))
print(df_list.get(all_files[0]))

88
88
       time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  var_rss23
0       250      42.00       0.00      18.00       0.00      11.33       0.94
1       500      42.75       0.43      16.75       1.79      18.25       0.43
2       750      42.50       0.50      16.75       0.83      19.00       1.22
3      1000      43.00       0.82      16.25       0.83      18.00       0.00
4      1250      43.67       0.47      12.75       0.83      19.00       0.82
..      ...        ...        ...        ...        ...        ...        ...
474  118750      44.25       0.83      16.25       1.30      24.00       0.00
475  119000      44.00       0.71      11.00       1.00      24.00       0.00
476  119250      44.25       0.43      10.25       1.30      22.75       0.83
477  119500      43.75       0.43       7.33       2.87      21.50       0.50
478  119750      44.50       0.50       3.00       1.22      21.50       0.50

[479 rows x 7 columns]


The first time I run the above code I got the error:
```
Error reading file ../../data/AReM/cycling/dataset14.csv: Error tokenizing data. C error: Expected 7 fields in line 485, saw 8

Error reading file ../../data/AReM/cycling/dataset9.csv: Error tokenizing data. C error: Expected 7 fields in line 485, saw 8
```

I cleaned the data by deleting the extra comma at the end of these two files

Now all 88 instances are collectively stored in `df_list` in their original format.

### (b) Test and Train Data

Rubrics says: Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data and other datasets as train data.

I have to get all the names for the test set data.

In [7]:
# the names of all the test datasets.
rootdir = '../../data/AReM/'
test_files = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        # Check if the file is a CSV (because there are pdf files exist)
        if (file.endswith('t1.csv') or
            file.endswith('t2.csv') or
            file.endswith('t3.csv')):
            if (not file.endswith("bending1/dataset3.csv")
                and not file.endswith("bending2/dataset3.csv")):
                test_files.append(os.path.join(subdir, file))

test_files.remove("../../data/AReM/bending1/dataset3.csv")
test_files.remove("../../data/AReM/bending2/dataset3.csv")
print(len(test_files))

19


In [8]:
# store datasets used to generate testset into df_test_ori
df_test_ori = []
for file in test_files:
    df_test_ori.append(df_list[file])

print("number of test datasets:", len(df_test_ori))
print("the first test dataset: \n", df_test_ori[0])

# store training datasets into df_train_ori
df_train_ori = []
for file in all_files:
    if file not in test_files:
        df_train_ori.append(df_list[file])

print("number of training datasets:", len(df_train_ori))
print("the first training dataset: \n", df_train_ori[0])

number of test datasets: 19
the first test dataset: 
        time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  var_rss23
0       250      39.25       0.43      23.00       0.00       33.0        0.0
1       500      39.25       0.43      23.25       0.43       33.0        0.0
2       750      39.50       0.50      23.00       0.71       33.0        0.0
3      1000      39.50       0.50      24.00       0.00       33.0        0.0
4      1250      39.25       0.43      24.00       0.00       33.0        0.0
..      ...        ...        ...        ...        ...        ...        ...
474  118750      43.33       0.47      25.00       0.00       30.0        0.0
475  119000      43.50       0.50      25.50       0.50       30.0        0.0
476  119250      43.50       0.50      24.75       0.43       30.0        0.0
477  119500      43.50       0.50      24.33       0.47       30.0        0.0
478  119750      43.50       0.50      24.25       0.43       30.0        0.0

[479 rows

### (c) Feature Extraction

#### i. Research

My research starts with the definition of time series and its most used models (like ARMA, ARIMA and VAR). (Reference: [wikipedia: Time Series](https://en.wikipedia.org/wiki/Time_series#:~:text=In%20mathematics%2C%20a%20time%20series,equally%20spaced%20points%20in%20time.) and [wikipedia: Time Domain](https://en.wikipedia.org/wiki/Time_domain#:~:text=Time%20domain%20refers%20to%20the,data%2C%20with%20respect%20to%20time.)) Time Series has basic patterns to be captured in terms of 'trend', 'seasonality', 'cycle' and 'variation', but these are not features what we are looking for here.

After getting an idea of how Time Series Analysis is done, I started researching for its Machine Learning strategies. (Reference: [Time Series Forecasting with Machine Learning - youtube.com](https://www.youtube.com/watch?v=_ZQ-lQrK9Rg)) The features being selected usually include: 

**Simple statistical features:**

* Means in each of the 𝑑 dimensions
* Standard deviations of the 𝑑 dimensions
* Skewness, Kurtosis and Higher order moments of the 𝑑 dimensions
* Maximum and Minimum values

**Time serie analysis related features:**

* The 𝑑×𝑑−1 Cross-Correlations between each dimension and the 𝑑 Auto-Correlations
* Orders of the autoregressive (AR), integrated (I) and moving average (MA) part of an estimated ARIMA model
* Parameters of the AR part
* Parameters of the MA part

(Reference: [Stack Exchange: features for time series classification](https://stats.stackexchange.com/questions/50807/features-for-time-series-classification))

* Average Amplitude Chang
* Approximate Entropy
* Autoregressive Coefficients
* Box-Counting Dimension
* Cepstral Coefficients
* Difference absolute stdev value
* Detrended fluctation analysis
* Higuchi's fractal dimension
* Histogram
* Integral abs. value
* Standard deviation
* Kurtosis
* Log detector
* Modified mean abs val 1
* Modified mean abs val 2
* Meanabs deviation
* Mean abs val. slope
* Max. Fractal length
* Multiple hamming windows
* Multiple trapezoida windows
* Myopulse percentage rate
* Root mean square
* Sample Entropy
* Skewness
* Slope sign Change
* Simple square integral
* Absolute temporal moment
* Variance
* Variance Fracta Dimension
* v-order
* Willison amplitude
* Waveform length
* Zero Crossing
* Mean
(Reference: [Comparison of Different Time and Frequency Domain Feature Extraction Methods on Elbow
Gesture’s EMG](https://revistia.org/files/articles/ejis_v2_i3s_16/Cemil.pdf#:~:text=Mean%20is%20the%20most%20common,sample%20length%20of%20the%20signal.&text=Variance%20is%20also%20most%20common%20statistical%20method%20for%20time%20domain%20feature%20extraction.&text=AR%20coefficients%20are%20popular%20feature%20extraction%20method%20for%20biological%20signals.))

**Time-domain specifications (TDS)** include:
the lower and/or upper bounds of the quantities of the time response such as the first peak time, maximum peak time, rise time, maximum overshoot, maximum undershoot, setting time, and steady-state error.

(Reference: [Design of the Second-Order Controller by Time-Domain Objective Functions Using Cuckoo Search  DOI: 10.5772/intechopen.89832](https://www.intechopen.com/chapters/69848))

#### ii. Extraction

The features used here is as specified in the rubrics in hw3 sheet: 

minimum, maximum, mean, median, standard deviation, first quartile, and third quartile

**References**:

[python statistics module Doc](https://docs.python.org/3/library/statistics.html)

[How to add one row in an existing Pandas DataFrame?](https://www.geeksforgeeks.org/how-to-add-one-row-in-an-existing-pandas-dataframe/)

In [9]:
# setup the feature names for new dataset we want
output_features = []
original_features = df_b1_1.columns # get feature names in original datset
original_features = original_features.delete(0)
# print(original_features)
idx = 0
for original_feature in original_features:
    idx += 1
    if original_feature != 'time':
        min_name = f'min_{idx}_{original_feature}'
        output_features.append(min_name)
        max_name = f'max_{idx}_{original_feature}'
        output_features.append(max_name)
        mean_name = f'mean_{idx}_{original_feature}'
        output_features.append(mean_name)
        median_name = f'median_{idx}_{original_feature}'
        output_features.append(median_name)
        std_name = f'std_{idx}_{original_feature}'
        output_features.append(std_name)
        fstQ_name = f'fstQ_{idx}_{original_feature}'
        output_features.append(fstQ_name)
        trdQ_name = f'trdQ_{idx}_{original_feature}'
        output_features.append(trdQ_name)

print(len(output_features))
print(output_features[0])

# Note: here I expilicitly write the name of the 6 time series names into the feature names we are going to use, just for easy understanding.


42
min_1_avg_rss12


In [10]:
def calculate_feature_values(df):
    data_point = []
    for feature in original_features:
        series = np.array(df[feature])
        series_clean = [x for x in series if not pd.isna(x)]
        data_point.append(min(series_clean))
        data_point.append(max(series_clean))
        # reference: https://docs.python.org/3/library/statistics.html
        mean = statistics.mean(series_clean)
        data_point.append(mean)
        median = statistics.median(series_clean)
        data_point.append(median)
        stdev = statistics.stdev(series_clean)
        data_point.append(stdev)
        quartile = statistics.quantiles(series_clean, n=4)
        data_point.append(quartile[0])
        data_point.append(quartile[2])
    
    return data_point

# # check:
# print("a dataset: \n", df_test_ori[0])
# print("\n\na column = a time series: \n", df_test_ori[0].iloc[:, 1])
# print("\n\nlength of a datapoint: \n", len(calculate_feature_values(df_test_ori[0])))

# create empty dataset with fixed column names
# for testset
df_test = pd.DataFrame([],  columns = output_features)
# loop through all original test dataframes 
for df_ori in df_test_ori:
    series = calculate_feature_values(df_ori)
    df_test.loc[len(df_test.index)] = series
    # reference: https://www.geeksforgeeks.org/how-to-add-one-row-in-an-existing-pandas-dataframe/

# for training set
df_train = pd.DataFrame([],  columns = output_features)
# loop through all original training dataframes 
for df_ori in df_train_ori:
    series = calculate_feature_values(df_ori)
    df_train.loc[len(df_train.index)] = series

# check
# df_test.head()
df_train.head()

,min_1_avg_rss12,max_1_avg_rss12,mean_1_avg_rss12,median_1_avg_rss12,std_1_avg_rss12,fstQ_1_avg_rss12,trdQ_1_avg_rss12,min_2_var_rss12,max_2_var_rss12,mean_2_var_rss12,...,std_5_avg_rss23,fstQ_5_avg_rss23,trdQ_5_avg_rss23,min_6_var_rss23,max_6_var_rss23,mean_6_var_rss23,median_6_var_rss23,std_6_var_rss23,fstQ_6_var_rss23,trdQ_6_var_rss23
0,36.25,48.00,43.973236,44.50,1.617545,43.33,44.67,0.0,1.50,0.413987,...,3.289399,20.50,23.75,0.0,2.96,0.556472,0.49,0.487674,0.00,0.83
1,37.00,48.00,43.459562,43.25,1.383869,42.50,45.00,0.0,1.58,0.376159,...,2.454845,22.25,24.00,0.0,5.26,0.679102,0.50,0.623071,0.43,0.87
2,33.00,47.75,42.169061,43.50,3.666929,39.00,45.00,0.0,3.00,0.696514,...,3.853317,30.33,36.33,0.0,2.18,0.612088,0.50,0.523923,0.00,1.00
3,33.00,45.75,41.672526,41.75,2.242551,41.33,42.75,0.0,2.83,0.536117,...,2.413538,28.33,31.25,0.0,1.79,0.384092,0.43,0.389176,0.00,0.50
4,35.00,47.40,43.958580,44.33,1.557897,43.00,45.00,0.0,1.70,0.425658,...,1.985273,35.40,36.50,0.0,1.79,0.494322,0.43,0.513547,0.00,0.94


#### iii. Standard Deviation

**References**:

[convert  numpy dtypes to native Python types](https://www.w3resource.com/python-exercises/numpy/basic/numpy-basic-exercise-41.php)

[Calculating Confidence Intervals with Bootstrapping](https://towardsdatascience.com/calculating-confidence-interval-with-bootstrapping-872c657c058d)

In [11]:
# reference: https://towardsdatascience.com/calculating-confidence-interval-with-bootstrapping-872c657c058d

def bootstrap_ci(data, num_samples=10000, ci=0.90):
    # Create an empty array to store replicates
    bs_replicates = np.empty(num_samples)

    # Create bootstrap replicates as much as num_samples
    for i in range(num_samples):
        # Create a bootstrap sample
        bs_sample = np.random.choice(data,size=len(data))
        # Get bootstrap replicate and append to bs_replicates
        bs_replicates[i] = np.std(bs_sample)
    
    half_alpha = (1 - ci) / 2   # alpha / 2 = 0.05 in this case
    result = np.percentile(bs_replicates, [half_alpha * 100, (1 - half_alpha) * 100])
    return result

# check correctness of the function
print(bootstrap_ci(df_train["min_1_avg_rss12"]))


[7.48899641 9.94758056]


In [12]:
# Calculate standard deviation and bootstrap CI for each feature
results = []

for column in df_train.columns:
    std_dev = df_train[column].std().item()  # calculate standard deviation
    ci_low, ci_high = bootstrap_ci(df_train[column])
    ci_range = ci_high - ci_low
    results.append({
        'Standard Deviation': round(std_dev, 2),
        'CI': round(ci_range.item(), 2),
        'CI Lower': round(ci_low.item(), 2),
        'CI Upper': round(ci_high.item(), 2),
    })

# Display the results
# for result in results:
#     print(result)

In [13]:
# Convert results to a DataFrame for easy viewing
results_df = pd.DataFrame(results, index=df_train.columns)
results_df

,Standard Deviation,CI,CI Lower,CI Upper
min_1_avg_rss12,8.79,2.45,7.48,9.93
max_1_avg_rss12,4.43,2.13,3.24,5.37
mean_1_avg_rss12,4.92,1.08,4.30,5.38
median_1_avg_rss12,4.96,1.16,4.29,5.45
std_1_avg_rss12,1.76,0.40,1.54,1.93
fstQ_1_avg_rss12,5.73,1.04,5.12,6.16
trdQ_1_avg_rss12,4.78,1.60,3.90,5.49
min_2_var_rss12,0.00,0.00,0.00,0.00
max_2_var_rss12,5.15,0.86,4.63,5.49
mean_2_var_rss12,1.60,0.33,1.39,1.73


#### iv. Select Features

**reference**:

claudi.ai - chat screenshot [here](./Chat_link.pdf)

The generative AI gave me a direction of how to estimate and choose the best features. My own thinking is as follows.

We have two main criteria here: standard deviation and confidence interval. 

Standard deviation ususally refelcts how scattered the data points are, in the scopre of the whole dataset. Bigger in value of standard deviation indicates more differences and major movements of human activities captured by the signals. This might contain information that reflects diversity aspect of the data.

To analyse confidence interval, we should consider its definition. If a feature has smaller confidence interval, it means our estimates are more precise.

Taking the relation of these two values into consideration, the confidence interval is calculated using the standard error. Therefore, the narrower the width of confidence interval is compared to standard deviation, the more reliable and representive this feature is.

We want to find features with bigger standard deviation, and a confidence interval comparatively smaller then its standard deviation.

Below are the code I wrote to evaluate the importance of each feature.

In [14]:
# sort features by std(descding) and ci(ascending)
results_sorted_by_std = results_df.sort_values(by='Standard Deviation', ascending=False)
results_sorted_by_ci = results_df.sort_values(by='CI')

# initialize a datafreame to show importance
initial = {"min_cnt": 0, "max_cnt": 0, "mean_cnt": 0, "median_cnt": 0,
        "std_cnt": 0, "fstQ_cnt": 0, "trdQ_cnt": 0}
importance_df = pd.DataFrame(initial, index=[0])
importance_df

# count appearance
def count_appearance(top_num):
    min_cnt = max_cnt = mean_cnt = median_cnt = std_cnt = fstQ_cnt = trdQ_cnt = 0

    # weight = 42 - top_num
    weight = 10 / (top_num +1)  # My weighting algorithm

    top_std_features = results_sorted_by_std.head(top_num).index.tolist()
    top_ci_features = results_sorted_by_ci.head(top_num).index.tolist()
    for list in [top_std_features, top_ci_features]:
        for feature in list:
            if 'min' in feature: min_cnt += weight
            elif 'max' in feature: max_cnt += weight
            elif 'mean' in feature: mean_cnt += weight
            elif 'median' in feature: median_cnt += weight
            elif 'std' in feature: std_cnt += weight
            elif 'fstQ' in feature: fstQ_cnt += weight
            elif 'trdQ' in feature: trdQ_cnt += weight

    for feature in results_df.index.tolist():
        weight = results_df.loc[feature, 'Standard Deviation'].item() - results_df.loc[feature, 'CI'].item()
        if 'min' in feature: min_cnt += weight
        elif 'max' in feature: max_cnt += weight
        elif 'mean' in feature: mean_cnt += weight
        elif 'median' in feature: median_cnt += weight
        elif 'std' in feature: std_cnt += weight
        elif 'fstQ' in feature: fstQ_cnt += weight
        elif 'trdQ' in feature: trdQ_cnt += weight

    # result = {"min_cnt": min_cnt, "max_cnt": max_cnt, "mean_cnt": mean_cnt, "median_cnt": median_cnt,
    #         "std_cnt": std_cnt, "fstQ_cnt": std_cnt, "trdQ_cnt": trdQ_cnt}
    result = [min_cnt, max_cnt, mean_cnt, median_cnt, std_cnt, std_cnt, trdQ_cnt]
    # print(result)
    importance_df.loc[len(importance_df.index)] = result
    return result

for i in range(6, 42, 6):   # can be modified to see difference
    count_appearance(i)

importance_df.sum()

min_cnt        87.156862
max_cnt       116.838765
mean_cnt       89.416178
median_cnt     87.939588
std_cnt        39.164550
fstQ_cnt       39.164550
trdQ_cnt       89.586239
dtype: float64

From the above output, the best 3 features are **max, trdQ, mean**. By changing the range in the above code, **min** and **median** can also be an alternative to **mean**.

## 2. ISLR 3.7.4

[Question](./ISLR%203.7.4.jpg)

### (a) Linear Train

**Answer**: 

Most cases, we expect the training RSS of the **cubic** reggression to be lower than the linear one. (In rare cases the two can be same depending on the irredducible error of the training set. Almost not possible)

**Reason**: 

Start with the assumption we make here: the true relationship is linear. This means, the linear regression is more likely to show the real underlying relationship. Howerver, because we are comparing the **traing** RSS for the two regression. It is obvious that the cubic regression is more complex and curvy, having the ability to fit better with the given training data. Given the true relationship is linear, the added higher-order terms are unnecessary yet can still reduce training RSS. It usually cause overfitting.

There are also rare cases that the training RSS of the linear regression the same with that of the cubic. This only happens when ∊, the irreducible error is very small to almost zero, or they cancle out perfectly so that the RSS for the two just the same.

The truth is, the cubic terms being added may be redundant for the test set, but it will work better on the training set because of overfitting.


### (b) Linear Test

**Answer**:

Most cases, we expect the test RSS of the **linear** reggression to be lower than the cubic one.

**Reason**: 

Same as above, start with the assumption we make here: the true relationship is linear. The linear regression is more likely to show the real underlying relationship. Here we are comparing the test RSS, which introduces new errors, and therefore better reflects the model's generalization. 

The cubic regression however, is more sensitive to errors for having added unnecessary terms. Consequently, its test RSS is higher.


### (c) Not Linear Train

**Answer**: In most cases, the cubic regression will likely have lower RSS in both the training set and test set compared to linear regression. However, the degree of difference may vary.

**Reason**: 

Same as above, start with the assumption we make here: the true relationship is not linear, but more complex, regardless we don't know how far it is from linear. Because the cubic regression is more flexible, it can better reflect the complexity of the underlying patterns that deviate from linearity. 

If the true relationship is even slightly non-linear , such as quadratic, the cubic regression can do better by finding the coefficient ß2 to fit data more closely. 

As for the linear model, being constrained to a straight line, cannot adapt to these non-linear patterns. Observe the graph for **explanatory power**, the *unexplained variation* for the linear will be higher than that of the cubic, resulting in higher unexplained variation and thus higher training RSS.


### (d) Not Linear Testing

**Answer**: 

In most cases, the cubic regression will likely have lower RSS in both the training set and test set compared to linear regression.

**Reason**: 

Though the cubic regression will tend to have lower test RSS, the difference may be less exaggerated than in the training set.If the true relationship is moderately non-linear: The cubic model will generally outperform the linear model on test data, as it can better find the underlying pattern. The linear model just cannot capture non-linear trends, and will lead to consistently higher RSS.

If the true relationship is slightly non-linear: The cubic model may still have lower RSS, but the difference could be smaller. In some cases, the RSS values might be similar for both models. This can happen if the cubic model is slightly overfitting the training data, causing its performance on the test set to be closer to that of the linear model.

If the true relationship is more far from linear, such as cubic or more complex, the cubic model will likely have significantly lower RSS on both training and test sets, as the linear model simply cannot capture the curvy shape of the data.

To conclude, the relative performance on the test set depends on how well the cubic model matches the true underlying relationship, balancing the trade-off between bias and variance.

## 3. ISLR 3.7.3 - Extra Practice 

## 4. ISLR 3.7.5 - Extra Practice 

# References:

1. Python: 

    [Iterating through directories with Python](https://stackoverflow.com/questions/19587118/iterating-through-directories-with-python)

    [Read multiple csv files from multiple folders in Python](https://stackoverflow.com/questions/72663553/read-multiple-csv-files-from-multiple-folders-in-python)

    [Pandas Doc for read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)

    [python statistics module Doc](https://docs.python.org/3/library/statistics.html)

    [How to add one row in an existing Pandas DataFrame?](https://www.geeksforgeeks.org/how-to-add-one-row-in-an-existing-pandas-dataframe/)

    [convert  numpy dtypes to native Python types](https://www.w3resource.com/python-exercises/numpy/basic/numpy-basic-exercise-41.php)

2. Time Series

    [wikipedia: Time Series](https://en.wikipedia.org/wiki/Time_series#:~:text=In%20mathematics%2C%20a%20time%20series,equally%20spaced%20points%20in%20time.)

    [Stack Exchange: features for time series classification](https://stats.stackexchange.com/questions/50807/features-for-time-series-classification)

    [wikipedia: Time Domain](https://en.wikipedia.org/wiki/Time_domain#:~:text=Time%20domain%20refers%20to%20the,data%2C%20with%20respect%20to%20time.)

    [Design of the Second-Order Controller by Time-Domain Objective Functions Using Cuckoo Search  DOI: 10.5772/intechopen.89832](https://www.intechopen.com/chapters/69848)

    [Time Series Forecasting with Machine Learning - youtube.com](https://www.youtube.com/watch?v=_ZQ-lQrK9Rg)

3.  [Calculating Confidence Intervals with Bootstrapping](https://towardsdatascience.com/calculating-confidence-interval-with-bootstrapping-872c657c058d)

4. AI: claudi.ai - chat screenshot [here](./Chat_link.pdf)
